In [17]:
!wget https://github.com/doctorai-in/solar-energy-prediction/raw/main/datasets.zip

--2021-01-16 07:11:32--  https://github.com/doctorai-in/solar-energy-prediction/raw/main/datasets.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/doctorai-in/solar-energy-prediction/main/datasets.zip [following]
--2021-01-16 07:11:32--  https://raw.githubusercontent.com/doctorai-in/solar-energy-prediction/main/datasets.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152849 (149K) [application/zip]
Saving to: ‘datasets.zip.1’

datasets.zip.1      100%[===================>] 149.27K  --.-KB/s    in 0.03s   

2021-01-16 07:11:32 (4.67 MB/s) - ‘datasets.zip.1’ saved [152849/152849]



In [18]:
!unzip datasets.zip 

Archive:  datasets.zip
replace datasets/hourly/weather_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: datasets/hourly/weather_train.csv  
  inflating: datasets/hourly/weather_test.csv  
  inflating: datasets/hourly/weather_dev.csv  


In [19]:
import numpy as np
import math
import matplotlib as mpl
from matplotlib.image import imread
from random import randint

import theano
import keras
import pandas

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras import optimizers
import keras.utils
import keras.layers
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import r2_score
import copy
import csv

In [20]:
mpl.use('Agg')
import matplotlib.pyplot as plt

DATASET_PATH = "./datasets/hourly/"

In [21]:
#Set y values of data to lie between 0 and 1
def normalize_data(dataset, data_min, data_max):
    data_std = (dataset - data_min) / (data_max - data_min)
    test_scaled = data_std * (np.amax(data_std) - np.amin(data_std)) + np.amin(data_std)
    return test_scaled

#Import and pre-process data for future applications
def import_data(train_dataframe, dev_dataframe, test_dataframe):
    
    dataset = train_dataframe.values
    dataset = dataset.astype('float32')

    #Include all 12 initial factors (Year ; Month ; Hour ; Day ; Cloud Coverage ; Visibility ; Temperature ; Dew Point ;
    #Relative Humidity ; Wind Speed ; Station Pressure ; Altimeter
    max_test = np.max(dataset[:,12])
    min_test = np.min(dataset[:,12])
    scale_factor = max_test - min_test
    max = np.empty(13)
    min = np.empty(13)

    #Create training dataset
    for i in range(0,13):
        min[i] = np.amin(dataset[:,i],axis = 0)
        max[i] = np.amax(dataset[:,i],axis = 0)
        dataset[:,i] = normalize_data(dataset[:, i], min[i], max[i])

    train_data = dataset[:,0:12]
    train_labels = dataset[:,12]

    # Create dev dataset
    dataset = dev_dataframe.values
    dataset = dataset.astype('float32')

    for i in range(0, 13):
        dataset[:, i] = normalize_data(dataset[:, i], min[i], max[i])

    dev_data = dataset[:,0:12]
    dev_labels = dataset[:,12]

    # Create test dataset
    dataset = test_dataframe.values
    dataset = dataset.astype('float32')

    for i in range(0, 13):
        dataset[:, i] = normalize_data(dataset[:, i], min[i], max[i])

    test_data = dataset[:, 0:12]
    test_labels = dataset[:, 12]

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels, scale_factor

In [22]:
#Construt and return Keras RNN model
def build_model(init_type = 'glorot_uniform', optimizer = 'adam', num_features = 12):
    model = Sequential()
    layers = [num_features, 64, 64, 1, 1]
    model.add(keras.layers.LSTM(
        layers[0],
        input_shape = (None, num_features),
        return_sequences=True))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.LSTM(
        layers[1],
        kernel_initializer = init_type,
        return_sequences=True
        #bias_initializer = 'zeros'
    ))
    model.add(keras.layers.Dropout(0.2))

    model.add(Dense(
        layers[2], activation='tanh',
        kernel_initializer=init_type,
        input_shape = (None, 1)
        ))
    model.add(Dense(
        layers[3]))

    model.add(Activation("relu"))

    #Alternative parameters:
    #momentum = 0.8
    #learning_rate = 0.1
    #epochs = 100
    #decay_rate = learning_rate / 100
    #sgd = keras.optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    #model.compile(loss="binary_crossentropy", optimizer=sgd)
    rms = keras.optimizers.RMSprop(lr=0.002, rho=0.9, epsilon=1e-08, decay=0.01)
    model.compile(loss="mean_squared_error", optimizer=optimizer)

    return model

In [23]:
#Save output predictions for graphing and inspection
def write_to_csv(prediction, filename):
    print("Writing to CSV...")
    with open(filename, 'w') as file:
        for i in range(prediction.shape[0]):
            file.write("%.5f" % prediction[i][0][0])
            file.write('\n')
    print("...finished!")

# Return MSE error values of all three data sets based on a single model
def evaluate(model, X_train, Y_train, X_dev, Y_dev, X_test, Y_test, scale_factor):
    scores = model.evaluate(X_train, Y_train, verbose = 0) * scale_factor * scale_factor
    print("train: ", model.metrics_names, ": ", scores)
    scores = model.evaluate(X_dev, Y_dev, verbose = 0) * scale_factor * scale_factor
    print("dev: ", model.metrics_names, ": ", scores)
    scores = model.evaluate(X_test, Y_test, verbose = 0) * scale_factor * scale_factor
    print("test: ", model.metrics_names, ": ", scores)

# Calculate MSE between two arrays of values
def mse(predicted, observed):
    return np.sum(np.multiply((predicted - observed),(predicted - observed)))/predicted.shape[0]

In [24]:
def main():
    #plt.switch_backend('tkAgg')

    #Import test data (6027, 13)
    train_dataframe = pandas.read_csv(DATASET_PATH + 'weather_train.csv', sep=";", engine='python', header = None)
    dev_dataframe = pandas.read_csv(DATASET_PATH + 'weather_dev.csv', sep=";", engine='python', header = None)
    test_dataframe = pandas.read_csv(DATASET_PATH + 'weather_test.csv', sep=";", engine='python', header = None)
    train_data, train_labels, dev_data, dev_labels, test_data, test_labels, scale_factor = import_data(train_dataframe, dev_dataframe, test_dataframe)
    from IPython.display import display
    display(train_dataframe.head(10))


    time_steps = 1
    assert(train_data.shape[0] % time_steps == 0)

    X_train = np.reshape(train_data, (train_data.shape[0] // time_steps, time_steps, train_data.shape[1]))
    X_dev = np.reshape(dev_data, (dev_data.shape[0] // time_steps, time_steps, dev_data.shape[1]))
    X_test = np.reshape(test_data, (test_data.shape[0] // time_steps, time_steps, test_data.shape[1]))
    Y_train = np.reshape(train_labels, (train_labels.shape[0] // time_steps, time_steps, 1))
    Y_dev = np.reshape(dev_labels, (dev_labels.shape[0] // time_steps, time_steps, 1))
    Y_test = np.reshape(test_labels, (test_labels.shape[0] // time_steps, time_steps, 1))

    model = build_model('glorot_uniform', 'adam')

    #Standard vanilla LSTM model

    model_fit_epochs = 100
    print("X_train shape: ",X_train.shape, " Y_train shape: ",Y_train.shape)

    model.fit(
        X_train, Y_train,
        batch_size = 16, epochs = model_fit_epochs)
    trainset_predicted = model.predict(X_train)
    devset_predicted = model.predict(X_dev)
    testset_predicted = model.predict(X_test)
    
    print(trainset_predicted.shape, Y_train.shape)
    
    print("Train r2_score: ", r2_score(trainset_predicted.reshape(-1, 1), Y_train.reshape(-1, 1)) )
    print("Dev r2_score: ", r2_score(devset_predicted.reshape(-1, 1), Y_dev.reshape(-1, 1)) )
    print("Test r2_score: ", r2_score(testset_predicted.reshape(-1, 1), Y_test.reshape(-1, 1)))

    print("Train MSE: ", mse(trainset_predicted, Y_train) * scale_factor * scale_factor)
    print("Dev MSE: ", mse(devset_predicted, Y_dev) * scale_factor * scale_factor)
    print("Test MSE: ", mse(testset_predicted, Y_test) * scale_factor * scale_factor)

    write_to_csv(trainset_predicted,'nn_trainset_prediction.csv')
    write_to_csv(devset_predicted,'nn_devset_prediction.csv')
    write_to_csv(testset_predicted, 'nn_testset_prediction.csv')


In [25]:
main()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,12,2,1,2017,1.00,1.85,6.97,6.90,98.24,10.92,29.18,29.97,449.25
1,9,7,2,2016,0.00,10.00,3.96,-0.61,70.20,17.96,29.12,29.91,3165.50
2,9,22,2,2016,0.04,10.00,2.99,-1.34,71.04,10.57,29.37,30.17,2637.00
3,8,3,10,2017,1.00,10.00,21.60,17.14,72.80,11.84,29.53,30.33,710.25
4,7,27,2,2017,1.00,10.00,2.75,0.70,85.40,6.64,29.37,30.17,680.50
5,6,25,5,2016,0.00,10.00,20.08,17.37,86.04,8.04,29.26,30.05,273.95
6,13,25,4,2017,0.00,10.00,26.22,9.53,32.68,19.68,28.85,29.63,4692.00
7,10,23,6,2016,0.53,10.00,30.03,24.38,69.92,9.48,29.09,29.88,3000.00
8,14,28,5,2017,0.73,10.00,25.62,16.38,55.28,17.20,28.97,29.76,3040.50
9,7,11,4,2016,0.45,7.97,10.10,8.58,88.27,10.18,29.10,29.89,297.75


X_train shape:  (6028, 1, 12)  Y_train shape:  (6028, 1, 1)
Epoch 1/100
377/377 [==============================] - 6s 4ms/step - loss: 0.0998
Epoch 2/100
377/377 [==============================] - 2s 4ms/step - loss: 0.0649
Epoch 3/100
377/377 [==============================] - 1s 4ms/step - loss: 0.0576
Epoch 4/100
377/377 [==============================] - 1s 4ms/step - loss: 0.0549
Epoch 5/100
377/377 [==============================] - 1s 4ms/step - loss: 0.0519
Epoch 6/100
377/377 [==============================] - 1s 4ms/step - loss: 0.0490
Epoch 7/100
377/377 [==============================] - 1s 4ms/step - loss: 0.0442
Epoch 8/100
377/377 [==============================] - 2s 4ms/step - loss: 0.0389
Epoch 9/100
377/377 [==============================] - 2s 4ms/step - loss: 0.0364
Epoch 10/100
377/377 [==============================] - 1s 4ms/step - loss: 0.0354
Epoch 11/100
377/377 [==============================] - 1s 4ms/step - loss: 0.0356
Epoch 12/100
377/377 [==============